In [ ]:
!unzip drive/MyDrive/data.zip -d /content/

In [ ]:
!mkdir Train
!mkdir Test

In [ ]:
import os
import shutil
import random

In [ ]:
def split_data(source, train_dir, test_dir, split_ratio):
    for class_folder in os.listdir(source):
        class_path = os.path.join(source, class_folder)
        if os.path.isdir(class_path):
            files = os.listdir(class_path)
            random.shuffle(files)  # Shuffle the files to ensure randomness in the split
            split_index = int(len(files) * split_ratio)
            train_files = files[:split_index]
            test_files = files[split_index:]

            # Create class folders in train and test directories if they don't exist
            train_class_dir = os.path.join(train_dir, class_folder)
            test_class_dir = os.path.join(test_dir, class_folder)
            os.makedirs(train_class_dir, exist_ok=True)
            os.makedirs(test_class_dir, exist_ok=True)

            # Move files to train directory
            for file in train_files:
                src = os.path.join(class_path, file)
                dst = os.path.join(train_class_dir, file)
                shutil.copy(src, dst)

            # Move files to test directory
            for file in test_files:
                src = os.path.join(class_path, file)
                dst = os.path.join(test_class_dir, file)
                shutil.copy(src, dst)

In [ ]:
source_directory = "data"
train_directory = "Train"
test_directory = "Test"
split_ratio = 0.8  # 80% training data, 20% testing data

split_data(source_directory, train_directory, test_directory, split_ratio)


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Set the image size and batch size
IMAGE_SIZE = (128, 128)
BATCH_SIZE = 32

# Define the number of classes
NUM_CLASSES = 35  # For example, if you have 10 hand sign classes

# Load the pre-trained MobileNet model
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(*IMAGE_SIZE, 3))

# Add custom top layers for our task
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

# Create the full model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the base MobileNet layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    'Train',
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    'Test',
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical')

In [ ]:

# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=100,  # adjust number of epochs as needed
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE)

In [ ]:
# Save the trained model
model.save('hand_sign_model.h5')


In [ ]:
# Load the saved Keras model
model = tf.keras.models.load_model('hand_sign_model.h5')

# Convert the model to TFLite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TFLite model to a file
with open('hand_sign_model.tflite', 'wb') as f:
    f.write(tflite_model)
